## Создание сети через класс в Python

1. Рассмотрим ниже пример создания простого класса. 

In [ ]:
class Dog:
   
    # Class Variable
    animal = 'dog'            
   
    # The init method or constructor
    def __init__(self, breed, color):
     
        # Instance Variable    
        self.breed = breed
        self.color = color
    
# Objects of Dog class
Rodger = Dog("Pug", "brown")
Buzo = Dog("Bulldog", "black")
 
print('Rodger details:')  
print('Rodger is a', Rodger.animal)
print('Breed: ', Rodger.breed)
print('Color: ', Rodger.color)
 
print('\nBuzo details:')  
print('Buzo is a', Buzo.animal)
print('Breed: ', Buzo.breed)
print('Color: ', Buzo.color)
 
# Class variables can be accessed using class
# name also
print("\nAccessing class variable using class name")
print(Dog.animal)  

2. Добавление методов классу

In [ ]:
class Dog:
       
    # Class Variable
    animal = 'dog'     
       
    # The init method or constructor
    def __init__(self, breed='corgi'):
        # Instance Variable
        self.breed = breed            
   
    # Adds an instance variable 
    def setColor(self, color):
        self.color = color
       
    # Retrieves instance variable    
    def getColor(self):
        return self.color   
    
    def ask_for_food(self):
        print('Gimme food pease!')


Rodger = Dog("pug")
Rodger.setColor("brown")
print(Rodger.getColor()) 

Rodger.ask_for_food()

3. Наследование

In [ ]:
class Pet(Dog):
    def __init__(self, breed):
        super().__init__(breed)

    def bark(self):
        print('Bark-bark!')
    
    def ask_for_food(self):
        print('Excuse me sir, would you mind to provide me with a meal?')


pet = Pet("corgi")
pet.bark()

pet.setColor("brown")
print(pet.getColor())

pet.ask_for_food()


**Вопрос на внимательность:** Что можно улучшить в обоих классах?

### Нейронная сеть через Python класс

Ранее архитектура сети задавалась с помощью функции. Более удобный, гибкий и общепринятый подход для описания архитектуры сетей является использование классов в Python.

#### 1. Так делали раньше

In [ ]:
INPUT_SIZE = 37
HIDDEN_SIZE = 25
OUTPUT_SIZE = 4

def build_model():
    model = nn.Sequential(  
        # Добавляем в нашу модель первый слой из 25 нейронов
        nn.Linear(in_features=INPUT_SIZE, out_features=HIDDEN_SIZE),
        nn.ReLU(),
        
        # Добавляем ещё один слой из 25 нейронов
        nn.Linear(in_features=HIDDEN_SIZE, out_features=HIDDEN_SIZE),
        nn.ReLU(),
        
        # Выходной вектор на количество классов, получаем с помощью такого же линейного приеобразования,
        # как и предыдущие слои, но уже на нужное количество выходных нейронов (т.е. классов)
        nn.Linear(in_features=HIDDEN_SIZE, out_features=OUTPUT_SIZE),
        nn.Softmax()
    )

    return model

#### 2. Так делаем теперь

In [ ]:
import torch
import torch.nn as nn

class TwoLayerNet(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        """
        TwoLayerNet наследуется от nn.Module и тем самым полчаем возможность
        переопределять методы класса.
        В конструктуре создаем слои (обучаемые веса) и другие нужные перменные/функции,
        которые нужны для модели
        """
        super().__init__()  # super(TwoLayerNet, self).__init__()
        self.linear1 = torch.nn.Linear(input_size, hidden_size)
        self.linear2 = torch.nn.Linear(hidden_size, output_size)

    def forward(self, x):
        """
        Метод forward отвечает за прямое распростронение модели, 
        поэтому данный метод нужно переопределять обязательно, 
        чтобы задать логику прямого распростронения. 
        Именно в этот момент начинает строится динамический граф
        """
        h_relu = self.linear1(x).clamp(min=0)
        y_pred = self.linear2(h_relu)
        
        return y_pred

Запустим стандартный цикл обучения нейронной сети с архитектурой, представленной выше. 

In [ ]:
BATCH_SIZE = 64
INPUT_SIZE = 1000
HIDDEN_SIZE = 100
OUTPUT_SIZE = 10

dtype = torch.float
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu') 

x = torch.randn(BATCH_SIZE, INPUT_SIZE, device=device, dtype=dtype)
y = torch.randn(BATCH_SIZE, OUTPUT_SIZE, device=device, dtype=dtype)

model = TwoLayerNet(INPUT_SIZE, HIDDEN_SIZE, OUTPUT_SIZE)

loss_fn = torch.nn.MSELoss(reduction='mean')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)

for t in range(500):
    y_pred = model(x)

    loss = loss_fn(y_pred, y)
    if t % 100 == 99:
        print(f'{str(t):^4s} {loss.item():.4f}')

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()